# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [1]:
!pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 883.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 4.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
!pip install transformers==4.28.1 pytorch-lightning==1.6.1

# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [3]:
%cd /gdrive/MyDrive/hjh_kita_directory/Github/kita_231026/m7_npl응용/qa

/gdrive/MyDrive/hjh_kita_directory/Github/kita_231026/m7_npl응용/qa


In [4]:
from arguments import QADeployArguments
args = QADeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-qa/epoch=0-val_loss=0.46.ckpt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [5]:
import torch
from transformers import BertConfig, BertForQuestionAnswering
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)
model = BertForQuestionAnswering(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [6]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [7]:
def inference_fn(question, context):
    if question and context:
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False, # 순수한 질문 텍스트만 토큰화 하겠다.
            truncation=True,
            max_length=args.max_query_length
       )
        inputs = tokenizer.encode_plus( # 질문과 텍스트를 함께 토큰화
            text=truncated_query,
            text_pair=context,
            truncation="only_second", # pair 문장이 주어졌을 경우 두번째 문장에 대해서만 truncation
            padding="max_length",
            max_length=args.max_seq_length,
            return_token_type_ids=True,
        )
        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})
            start_pred = outputs.start_logits.argmax(dim=-1).item() # 시작 토큰에 해당하는 로짓(확률값들)에서 가장 큰 인덱스
            end_pred = outputs.end_logits.argmax(dim=-1).item() # 마지막 토큰에 해당하는 로짓(확률값들)에서 가장 큰 인덱스
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])
    else:
        pred_text = ""
    return {
        'question': question,
        'context': context,
        'answer': pred_text,
    }

In [8]:
question='은행권과 금융당국이 지금 해야될 일은 무엇입니까?'
context='이 원장은 "고금리가 지속되는 가운데 경기하강 우려도 커지면서 서민경제가 점점 어려워지고 있다"며 "이럴 때일수록 그간 충실한 자금 중개 기능을 통해 우리 경제의 버팀목 역할을 해온 은행권과 함께 감독당국이 서민경제 안정에 기여할 수 있도록 적극적으로 노력해야 한다"고 당부했다.'
inference_fn(question,context)

{'question': '은행권과 금융당국이 지금 해야될 일은 무엇입니까?',
 'context': '이 원장은 "고금리가 지속되는 가운데 경기하강 우려도 커지면서 서민경제가 점점 어려워지고 있다"며 "이럴 때일수록 그간 충실한 자금 중개 기능을 통해 우리 경제의 버팀목 역할을 해온 은행권과 함께 감독당국이 서민경제 안정에 기여할 수 있도록 적극적으로 노력해야 한다"고 당부했다.',
 'answer': '적극적으로 노력'}

In [9]:
question='MAU가 무엇입니까?'
context='모바일 빅데이터 기업 아이지에이웍스 모바일인덱스 데이터에 따르면 유튜브는 지난해 12월 월간 활성 이용자 수(MAU) 4565만명으로 전체 1위를 차지하며 카카오톡(4554만명)을 처음으로 제쳤다. MAU란 한 달에 1번 이상 서비스를 사용한 이용자 수를 뜻한다.'
inference_fn(question,context)

{'question': 'MAU가 무엇입니까?',
 'context': '모바일 빅데이터 기업 아이지에이웍스 모바일인덱스 데이터에 따르면 유튜브는 지난해 12월 월간 활성 이용자 수(MAU) 4565만명으로 전체 1위를 차지하며 카카오톡(4554만명)을 처음으로 제쳤다. MAU란 한 달에 1번 이상 서비스를 사용한 이용자 수를 뜻한다.',
 'answer': '[CLS]'}